In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

In [3]:
msm.convert('traj.h5', to_form='traj_ions.h5', selection='molecule_type=="ion"')

'traj_ions.h5'

In [4]:
msm.info('traj.h5')

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_peptides,n_structures
file:h5,4720,1527,1519,3,1519,None,1511,7,1,20


In [5]:
msm.info('traj_ions.h5')

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_ions,n_structures
file:h5,7,7,0,1,0,None,7,20
